# Necessary Package Installations

In [4]:
pip install --upgrade spacy

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 11.3 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.1.7
    Uninstalling click-8.1.7:
      Successfully uninstalled click-8.1.7
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
redlines 0.4.2 requires click<9.0.0,>=8.1.3, but you have click 7.1.2 which is incompatible.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.7.4 which is incompatible.
medspacy 1.1.5 requires spacy<3.6,>=3.4.1, but you have spacy 3.7.4 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install --upgrade typer click

Defaulting to user installation because normal site-packages is not writeable
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.9/97.9 kB 16.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
  Attempting uninstall: typer
    Found existing installation: typer 0.3.2
    Uninstalling typer-0.3.2:
      Successfully uninstalled typer-0.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spacy 3.7.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.10.0 which is incompatible.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.7.4 which is incompatible.
weasel 0.3.4 requires typer<0.10.0,>=0.3.0, but you have typer 0.10.0 which is incompatible.
medspacy 1

In [1]:
pip install openai==0.28

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


# Necessary Imports

In [1]:
import json
from collections import Counter
import os
import spacy
import ast
from spacy import displacy
import requests
import time
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score, classification_report
import matplotlib.pyplot as plt
# from wordcloud import WordCloud, STOPWORDS
import pandas as pd
pd.set_option('display.max_rows', None)  # None means show all rows
pd.set_option('display.max_columns', None)  # None means show all columns
pd.set_option('display.width', None)  # None means auto-detect the display width
pd.set_option('display.max_colwidth', None)  # None means show full width of columns
import openai
openai.api_type = ""
openai.api_base = ""
openai.api_version = ""  #"2023-07-01-preview"
openai.api_key = ''

In [9]:
df_train_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv')
df_val_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv')
df_val_uw = pd.read_csv('Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv')

In [178]:
df_train_ms.shape

(2189, 9)

In [5]:
df_train_ms.sample(3)

,Unnamed: 0,Text ID,Text,Sentences,Error Flag,Error Sentence ID,Error Sentence,Corrected Sentence,Corrected Text
2092,2092,ms-train-2092,A 55-year-old patient is brought to the emerge...,0 A 55-year-old patient is brought to the emer...,1,7,Nitroglycerin was administered to the patient ...,Patient's aspirin dose is increased based on t...,A 55-year-old patient is brought to the emerge...
1387,1387,ms-train-1387,A 4-year old boy is brought to the emergency d...,0 A 4-year old boy is brought to the emergency...,0,-1,NaN,NaN,NaN
2123,2123,ms-train-2123,A previously healthy 8-year-old boy is brought...,0 A previously healthy 8-year-old boy is broug...,0,-1,NaN,NaN,NaN


# Creating a subset from these files (For testing purposes):

1. MEDIQA-CORR-2024-MS-TrainingData.csv
2. MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv
3. MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv

In [158]:
df_train_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv',index_col=0)
df_val_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv',index_col=0)
df_val_uw = pd.read_csv('Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv',index_col=0)
min_count = min(df_combined['Error Flag'].value_counts().min(), 50)
sampled_df = df_combined.groupby('Error Flag').sample(n=min_count, random_state=42)
sampled_df = sampled_df.sample(frac=1, random_state=42).reset_index(drop=True)

columns_to_merge = ["Text ID", "GPT_4_error_flag", "GPT_4_sentence_id", "GPT_4_Error_Sentence", "GPT_4_Corrected_Sentence"]
case_1_df = pd.read_csv('case_1.csv')
sampled_df = sampled_df.merge(case_1_df[columns_to_merge], on="Text ID", how="left")
sampled_df[["GPT_4_error_flag", "GPT_4_sentence_id", "GPT_4_Error_Sentence", "GPT_4_Corrected_Sentence"]] = sampled_df[["GPT_4_error_flag", "GPT_4_sentence_id", "GPT_4_Error_Sentence", "GPT_4_Corrected_Sentence"]].fillna('')

sampled_df.to_csv('sampled_ms_uw_df_ea_shared_task.csv',index=False)

df_train_ms: (2189, 8)
df_val_ms: (574, 8)
df_val_uw: (160, 8)
Combined dataframe: (2923, 8)


In [181]:
df_train_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv',index_col=0)
df_val_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv',index_col=0)
df_val_uw = pd.read_csv('Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv',index_col=0)

df_combined = pd.concat([df_train_ms, df_val_ms, df_val_uw], ignore_index=True)
print(f"df_train_ms: {df_train_ms.shape}\ndf_val_ms: {df_val_ms.shape}\ndf_val_uw: {df_val_uw.shape}\nCombined dataframe: {df_combined.shape}")

size_train_ms = df_train_ms.shape[0]
size_val_ms = df_val_ms.shape[0]
size_val_uw = df_val_uw.shape[0]
total_size = size_train_ms + size_val_ms + size_val_uw

proportion_train_ms = size_train_ms / total_size
proportion_val_ms = size_val_ms / total_size
proportion_val_uw = size_val_uw / total_size

samples_train_ms = int(round(proportion_train_ms * 300))
samples_val_ms = int(round(proportion_val_ms * 300))
samples_val_uw = 300 - samples_train_ms - samples_val_ms

def sample_balanced(df, n_samples):
    df_error = df[df["Error Flag"] == 1]
    df_no_error = df[df["Error Flag"] == 0]
    n_samples_each = n_samples // 2    
    if n_samples % 2 != 0:
        n_samples_each += 1
    sampled_error = df_error.sample(n=min(n_samples_each, len(df_error)), replace=False)
    sampled_no_error = df_no_error.sample(n=min(n_samples_each, len(df_no_error)), replace=False)
    return pd.concat([sampled_error, sampled_no_error]).sample(frac=1)  # Randomize order

sampled_train_ms = sample_balanced(df_train_ms, samples_train_ms)
sampled_val_ms = sample_balanced(df_val_ms, samples_val_ms)
sampled_val_uw = sample_balanced(df_val_uw, samples_val_uw)
final_df = pd.concat([sampled_train_ms, sampled_val_ms, sampled_val_uw])

final_df_balanced = final_df.groupby('Error Flag', group_keys=False).apply(lambda x: x.sample(min(len(x), 150))).sample(frac=1).reset_index(drop=True)
final_df_balanced.shape
final_df_balanced.to_csv('sampled_ms_uw_df_ea_shared_task_300_texts.csv',index=False)

df_train_ms: (2189, 8)
df_val_ms: (574, 8)
df_val_uw: (160, 8)
Combined dataframe: (2923, 8)


(202, 8)

In [176]:
final_df_balanced['Text ID'].apply(lambda x: '-'.join(x.split('-')[:2])).value_counts()  # Distribution of clinical text taken from the three sources

Text ID
ms-train    224
ms-val       60
uw-val       16
Name: count, dtype: int64

In [175]:
final_df_balanced['Error Flag'].value_counts(dropna=False)

Error Flag
0    150
1    150
Name: count, dtype: int64

In [10]:
df_train_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv',index_col=0)
df_val_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv',index_col=0)
df_val_uw = pd.read_csv('Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv',index_col=0)
total_size = len(df_train_ms) + len(df_val_ms) + len(df_val_uw)
ratios = [len(df_train_ms) / total_size, len(df_val_ms) / total_size, len(df_val_uw) / total_size]
samples_from_each = [int(200 * ratio) for ratio in ratios]
samples_from_each[-1] = 200 - sum(samples_from_each[:-1])
sampled_dfs = [df.sample(n) for df, n in zip([df_train_ms, df_val_ms, df_val_uw], samples_from_each)]
final_df = pd.concat(sampled_dfs)
final_df.shape
final_df.to_csv('sampled_ms_uw_df_ea_shared_task_200_texts.csv',index=False)

In [5]:
final_df['Text ID'].apply(lambda x: '-'.join(x.split('-')[:2])).value_counts()  # Distribution of clinical text taken from the three sources

Text ID
ms-train    149
ms-val       39
uw-val       12
Name: count, dtype: int64

In [11]:
final_df['Error Flag'].value_counts(dropna=False)

Error Flag
1    113
0     87
Name: count, dtype: int64

In [14]:
def extract_balanced_samples_optimized(df, n_samples=100):
    if "Error Flag" not in df.columns:
        raise ValueError("The DataFrame does not have the 'Error Flag' column.")
    n_per_class = n_samples // 2
    if df["Error Flag"].value_counts().min() < n_per_class:
        raise ValueError("Not enough samples in one or both classes to maintain balance.")
    sampled_df = pd.concat([
        df[df["Error Flag"] == 0].sample(n=n_per_class, random_state=1),
        df[df["Error Flag"] == 1].sample(n=n_per_class, random_state=1)
    ])
    sampled_df = sampled_df.sample(frac=1, random_state=2).reset_index(drop=True)
    return sampled_df

In [15]:
df_train_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-TrainingData.csv',index_col=0)
df_val_ms = pd.read_csv('Feb_1_2024_MS_Train_Val_Datasets/MEDIQA-CORR-2024-MS-ValidationSet-1-Full.csv',index_col=0)
df_val_uw = pd.read_csv('Feb_5_2024_UW_Validation_Set_Updated/MEDIQA-CORR-2024-UW-ValidationSet-1-Full_Feb.csv',index_col=0)

df_combined = pd.concat([df_train_ms, df_val_ms, df_val_uw], ignore_index=True)
extract_balanced_samples_optimized(df_combined, 100).to_csv('sampled_ms_uw_df_ea_shared_task_100_texts.csv',index=False)

# Common Evaluation Metric functions

In [ ]:
def print_metrics(file_name):
    df_results = pd.read_csv(file_name)
    print(f"Shape of dataframe:{df_results.shape}",flush=True)
    df_results = df_results.dropna(subset=['GPT_4_error_flag'])
    print(f"Shape of dataframe AFTER dropping NANs:{df_results.shape}",flush=True)
    df_results.reset_index(inplace=True)
    
    def convert_value(value):
        if value in ['yes', 'Yes']:
            return 1
        elif value in ['No', 'no']:
            return 0
        else:
            return np.nan
    
    df_results['GPT_4_error_flag'] = df_results['GPT_4_error_flag'].apply(convert_value)
    # df_results['GPT_4_error_flag'].value_counts(dropna=False)
    
    y_true = df_results['Error Flag']
    y_pred = df_results['GPT_4_error_flag']
    
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(4, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Predicted Error Flag (0: no error & 1: Error)')
    plt.ylabel('True Error Flag')
    plt.title('Confusion Matrix')
    plt.show()
    tn, fp, fn, tp = cm.ravel()
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    accuracy = accuracy_score(y_true, y_pred)
    print(f"True Positives (TP): {tp}")
    print(f"False Positives (FP): {fp}")
    print(f"True Negatives (TN): {tn}")
    print(f"False Negatives (FN): {fn}")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"F1 Score: {f1:.2f}")
    print(f"Accuracy: {accuracy:.2f}")

# Understanding the input Training dataset

In [203]:
df_train_ms.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2189 entries, 0 to 2188
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Unnamed: 0          2189 non-null   int64 
 1   Text ID             2189 non-null   object
 2   Text                2189 non-null   object
 3   Sentences           2189 non-null   object
 4   Error Flag          2189 non-null   int64 
 5   Error Sentence ID   2189 non-null   int64 
 6   Error Sentence      1219 non-null   object
 7   Corrected Sentence  1219 non-null   object
 8   Corrected Text      1219 non-null   object
dtypes: int64(3), object(6)
memory usage: 154.0+ KB


In [181]:
df_train_ms['Error Flag'].value_counts(dropna=False)

Error Flag
1    1219
0     970
Name: count, dtype: int64

In [183]:
df_train_ms['Error Sentence'][:10]

0    After reviewing imaging, the causal pathogen was determined to be Haemophilus influenzae.
1                                                                                          NaN
2                                                 Suspected of infection with Giardia lamblia.
3                                                                                          NaN
4                                               Causal organism is Staphylococcus epidermidis.
5                                                                                          NaN
6                                                  Patient is diagnosed with an EBV infection.
7                                                                                          NaN
8                                   Bartonella henselae was determined as the causal organism.
9                                                        Culture confirms Nocardia asteroides.
Name: Error Sentence, dtype: object

In [ ]:
error_text = ' '.join(df_train_ms['Error Sentence'].dropna().str.lower())
corrected_text = ' '.join(df_train_ms['Corrected Sentence'].dropna().str.lower())

wordcloud_error = WordCloud(stopwords=STOPWORDS, background_color='white').generate(error_text)
wordcloud_corrected = WordCloud(stopwords=STOPWORDS, background_color='white').generate(corrected_text)
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(wordcloud_error, interpolation='bilinear')
plt.title('Error Sentences')
plt.axis('off')

plt.subplot(1, 2, 2)
plt.imshow(wordcloud_corrected, interpolation='bilinear')
plt.title('Corrected Sentences')
plt.axis('off')
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
df_train_ms['Text'].apply(len).plot(kind='hist', bins=20, alpha=0.7)
plt.title('Frequency of Text Lengths')
plt.xlabel('Text Length')
plt.ylabel('Frequency')
plt.grid(axis='y', alpha=0.75)
plt.show()

# CASE 1: 7 examples given in Prompt with every feature 
## (Text, Sentences, Error flag, Error sentence id, error sentence, corrected sentence)

In [ ]:
'''
Types of errors taken care by this code:
1. Timeout: Request timed out: HTTPSConnectionPool(host='sweden-api.openai.azure.com', port=443): Read timed out. (read timeout=600)
2. InvalidRequestError: The response was filtered due to the prompt triggering Azure OpenAI's content management policy. Please modify your prompt and retry. To learn more about our content filtering policies please read our documentation: https://go.microsoft.com/fwlink/?linkid=2198766
3. Request timed out: HTTPSConnectionPool(host='sweden-api.openai.azure.com', port=443): Read timed out. (read timeout=600)
'''

df_test = pd.read_csv('March-26-2024-MEDIQA-CORR-Official-Test-Set.csv',encoding='ISO-8859-1')
print(f"Test set shape: {df_test.shape}\nColumns: {df_test.columns}\n")

df_test['gpt_4_1106_preview_error_flag'] = 0
df_test['gpt_4_1106_preview_sentence_id'] = 0
df_test['gpt_4_1106_preview_Error_Sentence'] = ''
df_test['gpt_4_1106_preview_Corrected_Sentence'] = ''

max_retries = 3
default_wait_time = 6  # Default wait time in seconds

for i, row in df_test.iterrows():
    print("#######################################################################")
    time.sleep(10)
    if i % 5 == 0:
        time.sleep(10)
        
    retry_count = 0
    while retry_count < max_retries:
        try:
            print(f"Currently accessing df_test id:{row['Text ID']}", flush=True)
            text = row['Text']
            clinical_sentences = row['Sentences']
            message_text = [{"role":"system","content":
                  f"""You are an AI trained in medical knowledge. Below are examples of clinical texts (delimited by triple quotes) and clinical texts divided into sentences (delimited by triple quotes) followed by an analysis of whether there is a diagnostic error and, if so, the sentence ID and the text span containing the error. The examples also show the corrected sentence that should have been the correct diagnosis.
                  Example 1:
                  Clinical Text: ```A 17-year-old boy comes to the physician because of body aches and sore throat for 1 week. He has no history of serious illness and takes no medications. He lives with his parents; they recently adopted a cat from an animal shelter. He is sexually active with one female partner, and they use condoms consistently. His temperature is 38.7 C (101.7 F), pulse is 99/min, and blood pressure is 110/72 mm Hg. Examination shows bilateral posterior cervical lymphadenopathy. The pharynx is red and swollen. Laboratory studies show:
                  Hemoglobin 15 g/dL
                  Leukocyte count 11,500/mm3
                  Segmented neutrophils 48%
                  Band forms 2%
                  Basophils 0.5%
                  Eosinophils 1%
                  Lymphocytes 45%
                  Monocytes 3.5%
                  When the patient's serum is added to a sample of horse erythrocytes, the cells aggregate together. The causal pathogen is cytomegalovirus.```
                  Clinical Text Sentences: ```0 A 17-year-old boy comes to the physician because of body aches and sore throat for 1 week.
                  1 He has no history of serious illness and takes no medications.
                  2 He lives with his parents; they recently adopted a cat from an animal shelter.
                  3 He is sexually active with one female partner, and they use condoms consistently.
                  4 His temperature is 38.7 C (101.7 F), pulse is 99/min, and
                  5 blood pressure is 110/72 mm
                  6 Hg.
                  7 Examination shows bilateral posterior cervical lymphadenopathy.
                  8 The pharynx is red and swollen.
                  9 Laboratory studies show:
                  10 Hemoglobin 15
                  11 g/dL
                  12 Leukocyte count 11,500/mm3 Segmented neutrophils 48%
                  13 Band forms 2%
                  14 Basophils 0.5%
                  15 Eosinophils 1%
                  16 Lymphocytes 45%
                  17 Monocytes 3.5%
                  18 When the patient's serum is added to a sample of horse erythrocytes, the cells aggregate together.
                  19 The causal pathogen is cytomegalovirus.```
                  Error: yes
                  Error Sentence ID: 19
                  Error Sentence Span: The causal pathogen is cytomegalovirus.
                  Corrected Sentence: The causal pathogen is Epstein-Barr virus.
                
                  Example 2:
                  Clinical Text: ```A previously healthy 18-year-old woman comes to the emergency department because of diarrhea and abdominal cramps since the previous evening. She has had around 3Ã¢â‚¬â€œ4 episodes of watery stools. She feels nauseous and has vomited twice. She recollects eating out 2 days ago. She has been on a vegan diet for 6 months. She takes no medications and has not traveled anywhere recently. Her temperature is 36.8 (98.2 F), pulse is 73/min, and blood pressure is 110/70 mm Hg. Examination shows dry mucous membranes. Abdominal examination is unremarkable. Norovirus was determined to be the casual organism.```
                  Clinical Text Sentences: ```0 A previously healthy 18-year-old woman comes to the emergency department because of diarrhea and abdominal cramps since the previous evening.
                  1 She has had around 3Ã¢â‚¬â€œ4 episodes of watery stools.
                  2 She feels nauseous and has vomited twice.
                  3 She recollects eating out 2 days ago.
                  4 She has been on a vegan diet for 6 months.
                  5 She takes no medications and has not traveled anywhere recently.
                  6 Her temperature is 36.8 (98.2 F), pulse is 73/min, and
                  7 blood pressure is 110/70 mm
                  8 Hg.
                  9 Examination shows dry mucous membranes.
                  10 Abdominal examination is unremarkable.
                  11 Norovirus was determined to be the casual organism.```
                  Error: No
                  Error Sentence ID: -1
                  Error Sentence Span: -1
                  Corrected Sentence: -1
                
                  Example 3:
                  Clinical Text: ```A 5-year-old boy is brought to the emergency department by his grandmother because of difficulty breathing. Over the past two hours, the grandmother has noticed his voice getting progressively hoarser and occasionally muffled, with persistent drooling. He has not had a cough. The child recently immigrated from Africa, and the grandmother is unsure if his immunizations are up-to-date. He appears uncomfortable and is sitting up and leaning forward with his chin hyperextended. His temperature is 39.5 C (103.1 F), pulse is 110/min, and blood pressure is 90/70 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 95%. Pulmonary examination shows inspiratory stridor and scattered rhonchi throughout both lung fields, along with poor air movement. Pharyngoscopy is ordered. ```
                  Clinical Text Sentences: ```0 A 5-year-old boy is brought to the emergency department by his grandmother because of difficulty breathing.
                  1 Over the past two hours, the grandmother has noticed his voice getting progressively hoarser and occasionally muffled, with persistent drooling.
                  2 He has not had a cough.
                  3 The child recently immigrated from Africa, and the grandmother is unsure if his immunizations are up-to-date.
                  4 He appears uncomfortable and is sitting up and leaning forward with his chin hyperextended.
                  5 His temperature is 39.5 C (103.1 F), pulse is 110/min, and blood pressure is 90/70 mm
                  6 Hg.
                  7 Pulse oximetry on room air shows an oxygen saturation of 95%.
                  8 Pulmonary examination shows inspiratory stridor and scattered rhonchi throughout both lung fields, along with poor air movement.
                  9 Pharyngoscopy is ordered.```
                  Error: yes
                  Error Sentence ID: 9
                  Error Sentence Span: Pharyngoscopy is ordered.
                  Corrected Sentence: Nasotracheal intubation is performed.
                
                  Example 4:
                  Clinical Text:```A 28-year-old man is brought to the emergency department after being struck by a car an hour ago as he was crossing the street. He did not lose consciousness. He is complaining of pain in his right arm, forehead, and pelvis. He also has the urge to urinate, but has been unable to do so since the accident. He takes no medications. His temperature is 37.1 C (98.9 F), pulse is 72/min, respirations are 18/min, and blood pressure is 118/82 mm Hg. There are abrasions over his scalp and face and a 1x3 cm area of ecchymosis above his right eye. Abdominal examination shows suprapubic tenderness. There is a scant amount of blood at the urethral meatus. There is no cervical spinal tenderness. Musculoskeletal examination shows tenderness and ecchymosis over his right distal forearm. An x-ray of the pelvis shows a fracture of the pelvic ramus. Retrograde urethrogram was then performed. A CT scan of the head and neck show no abnormalities.```
                  Clinical Text Sentences: ```0 A 28-year-old man is brought to the emergency department after being struck by a car an hour ago as he was crossing the street.
                  1 He did not lose consciousness.
                  2 He is complaining of pain in his right arm, forehead, and pelvis.
                  3 He also has the urge to urinate, but has been unable to do so since the accident.
                  4 He takes no medications.
                  5 His temperature is 37.1 C (98.9 F), pulse is 72/min, respirations are 18/min, and
                  6 blood pressure is 118/82 mm
                  7 Hg.
                  8 There are abrasions over his scalp and face and a 1x3 cm area of ecchymosis above his right eye.
                  9 Abdominal examination shows suprapubic tenderness.
                  10 There is a scant amount of blood at the urethral meatus.
                  11 There is no cervical spinal tenderness.
                  12 Musculoskeletal examination shows tenderness and ecchymosis over his right distal forearm.
                  13 An x-ray of the pelvis shows a fracture of the pelvic ramus.
                  14 Retrograde urethrogram was then performed.
                  15 A CT scan of the head and neck show no abnormalities.```
                  Error: No
                  Error Sentence ID: -1
                  Error Sentence Span: -1
                  Corrected Sentence: -1
                
                  Example 5:
                  Clinical Text: ```A 28-year-old man is brought to the emergency department after being struck by a car an hour ago as he was crossing the street. He did not lose consciousness. He is complaining of pain in his right arm, forehead, and pelvis. He also has the urge to urinate, but has been unable to do so since the accident. He takes no medications. His temperature is 37.1 C (98.9 F), pulse is 72/min, respirations are 18/min, and blood pressure is 118/82 mm Hg. There are abrasions over his scalp and face and a 1x3 cm area of ecchymosis above his right eye. Abdominal examination shows suprapubic tenderness. There is a scant amount of blood at the urethral meatus. There is no cervical spinal tenderness. Musculoskeletal examination shows tenderness and ecchymosis over his right distal forearm. An x-ray of the pelvis shows a fracture of the pelvic ramus. A CT scan of the head and neck show no abnormalities. IV pyelogram was then performed.```
                  Clinical Text Sentences: ```0 A 28-year-old man is brought to the emergency department after being struck by a car an hour ago as he was crossing the street.
                  1 He did not lose consciousness.
                  2 He is complaining of pain in his right arm, forehead, and pelvis.
                  3 He also has the urge to urinate, but has been unable to do so since the accident.
                  4 He takes no medications.
                  5 His temperature is 37.1 C (98.9 F), pulse is 72/min, respirations are 18/min, and
                  6 blood pressure is 118/82 mm
                  7 Hg.
                  8 There are abrasions over his scalp and face and a 1x3 cm area of ecchymosis above his right eye.
                  9 Abdominal examination shows suprapubic tenderness.
                  10 There is a scant amount of blood at the urethral meatus.
                  11 There is no cervical spinal tenderness.
                  12 Musculoskeletal examination shows tenderness and ecchymosis over his right distal forearm.
                  13 An x-ray of the pelvis shows a fracture of the pelvic ramus.
                  14 A CT scan of the head and neck show no abnormalities.
                  15 IV pyelogram was then performed.```
                  Error: yes
                  Error Sentence ID: 15
                  Error Sentence Span: IV pyelogram was then performed.
                  Corrected Sentence: Retrograde urethrogram was then performed.
                
                  Example 6:
                  Clinical Text: ```A previously healthy 25-year-old man comes to the physician because of a 4-day history of fever, joint and body pain, diffuse headache, and pain behind the eyes. This morning he noticed that his gums bled when he brushed his teeth. He returned from a backpacking trip to the Philippines 4 days ago. His temperature is 39.4 C (103.0 F). Physical examination shows a diffuse maculopapular rash. His leukocyte count is 3,200/mm3 and platelet count is 89,000/mm3. Further evaluation shows increased serum levels of a flavivirus. The patient is infected by the chikungunya virus.```
                  Clinical Text Sentences: ```0 A previously healthy 25-year-old man comes to the physician because of a 4-day history of fever, joint and body pain, diffuse headache, and pain behind the eyes.
                  1 This morning he noticed that his gums bled when he brushed his teeth.
                  2 He returned from a backpacking trip to the Philippines 4 days ago.
                  3 His temperature is 39.4 C (103.0 F).
                  4 Physical examination shows a diffuse maculopapular rash.
                  5 His leukocyte count is
                  6 3,200/mm3 and platelet count is 89,000/mm3.
                  7 Further evaluation shows increased serum levels of a flavivirus.
                  8 The patient is infected by the chikungunya virus.```
                  Error: yes
                  Error Sentence ID: 8
                  Error Sentence Span: The patient is infected by the chikungunya virus.
                  Corrected Sentence: The patient is infected by the Dengue virus.
                
                  Example 7:
                  Clinical Text: ```A previously healthy 25-year-old man comes to the physician with suspected Dengue viral infection because of a 4-day history of fever, joint and body pain, diffuse headache, and pain behind the eyes. This morning he noticed that his gums bled when he brushed his teeth. He returned from a backpacking trip to the Philippines 4 days ago. His temperature is 39.4 C (103.0 F). Physical examination shows a diffuse maculopapular rash. His leukocyte count is 3,200/mm3 and platelet count is 89,000/mm3. Further evaluation shows increased serum levels of a flavivirus.```
                  Clinical Text Sentences: ```0 A previously healthy 25-year-old man comes to the physician with suspected Dengue viral infection because of a 4-day history of fever, joint and body pain, diffuse headache, and pain behind the eyes.
                  1 This morning he noticed that his gums bled when he brushed his teeth.
                  2 He returned from a backpacking trip to the Philippines 4 days ago.
                  3 His temperature is 39.4 C (103.0 F).
                  4 Physical examination shows a diffuse maculopapular rash.
                  5 His leukocyte count is
                  6 3,200/mm3 and platelet count is 89,000/mm3.
                  7 Further evaluation shows increased serum levels of a flavivirus.```
                  Error: No
                  Error Sentence ID: -1
                  Error Sentence Span: -1
                  Corrected Sentence: -1
                
                  Now, you are given a new clinical text and Clinical Text Sentences delimited by triple quotes. Carefully evaluate and analyse the information presented in clinical text such as symptoms, clinical examination findings, patient history and other details. Determine if any of the given sentences contain a diagnostic error or not. If an error is present, identify the specific sentence ID (and the sentence with that ID) that contains this error and also correct this erroneous sentence based on the rest of clinical text. Use your knowledge and the context provided to make your assessment.

                  Clinical Text: ```{text}```
                  Clinical Text Sentences: ```{clinical_sentences}```
                
                  Provide the answers in JSON Format with the following keys:
                  Error, Error Sentence ID, Error Sentence, Corrected Sentence
                  """
                  }]

            completion = openai.ChatCompletion.create(engine="",messages = message_text,
                          temperature=0, max_tokens=800,
                          top_p=0.95, frequency_penalty=0,
                          presence_penalty=0, stop=None)
            
            message_content = completion["choices"][0]["message"]["content"].strip('```json\n').strip('\n```')
            message_content_json = json.loads(message_content)
            df_test.at[i, 'gpt_4_1106_preview_error_flag'] = message_content_json['Error']
            df_test.at[i, 'gpt_4_1106_preview_sentence_id'] = message_content_json['Error Sentence ID']
            df_test.at[i, 'gpt_4_1106_preview_Error_Sentence'] = message_content_json['Error Sentence']
            df_test.at[i, 'gpt_4_1106_preview_Corrected_Sentence'] = message_content_json['Corrected Sentence']
            df_test.to_csv('case_1_test_set_preds_temp.csv',index=False)
            break

        except openai.error.APIConnectionError as e:
            print(f"Timeout error encountered at index {i}, attempt {retry_count + 1}: {e}", flush=True)
            retry_count += 1
            time.sleep(default_wait_time)  # Wait before retrying

        except openai.error.InvalidRequestError as e:
            print(f"InvalidRequestError at index {i}: {e}", flush=True)
            break  # Skip to the next record

        except openai.error.RateLimitError as e:
            match = re.search(r"Please retry after (\d+) seconds", str(e))
            if match:
                wait_time = int(match.group(1)) + 1  # Add a 1-second buffer
            else:
                wait_time = default_wait_time
            print(f"RateLimitError at index {i}. Retrying after {wait_time} seconds.", flush=True)
            time.sleep(wait_time)
            retry_count += 1

        except openai.error.OpenAIError as e:
            print(f"Unexpected OpenAIError encountered at index {i}: {e}", flush=True)
            break  # Exit the retry loop on unexpected OpenAI error

        except Exception as e:
            print(f"Unexpected error encountered at index {i}: {e}", flush=True)
            break  # Exit the retry loop on any other unexpected error

df_test.to_csv('case_1_test_set_predictions.csv',index=False)
print("Final csv saved!!",flush=True)

# CASE 2: RAG for Positive Cases
And then we'll merge the cases predicted as 0 by GPT model with this predictions

In [2]:
# # Installations
# med_scipy_dict = {
#     'en_core_sci_scibert': 'https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_core_sci_scibert-0.5.4.tar.gz',
#     'en_ner_bc5cdr_md': 'https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.4/en_ner_bc5cdr_md-0.5.4.tar.gz'
# }
# for model_name, model_url in med_scipy_dict.items():
#     print(f"Installing {model_name}...")
#     os.system(f"pip install {model_url}")
# print("SciSpacy models have been installed.")

# Functions
def convert_value(value):
    if value in ['yes', 'Yes']:
        return 1
    elif value in ['No', 'no']:
        return 0
    else:
        return np.nan

def apply_models(dataframe, text_column):
    models = {
        'en_core_sci_scibert': spacy.load('en_core_sci_scibert'),
        'en_ner_bc5cdr_md': spacy.load('en_ner_bc5cdr_md')
    }
    for model_name in models.keys():
        dataframe[model_name] = None
    for index, row in dataframe.iterrows():
        text = row[text_column]
        for model_name, model in models.items():
            doc = model(text)
            extracted_info = [ent.text for ent in doc.ents]
            dataframe.at[index, model_name] = extracted_info
    return dataframe

def find_longest_keyword(keywords_list):
    if not keywords_list: # Return an empty string or a custom placeholder if the list is empty
        return ""
    return max(keywords_list, key=len)
    
def string_to_list(string):
    try:
        return ast.literal_eval(string)
    except (ValueError, SyntaxError):
        return []    # Return an empty list if the string cannot be evaluated

def remove_keywords(keywords_list):
    return [keyword for keyword in keywords_list if keyword.lower() not in keywords_to_remove]

def find_longest_common_keywords(row):
    keywords1, keywords2 = row['en_core_sci_scibert'], row['en_ner_bc5cdr_md']
    if not keywords1:
        return keywords2
    if not keywords2:
        return keywords1
    common_keywords = []
    for word1 in keywords1:
        for word2 in keywords2:
            if word1 in word2 or word2 in word1:
                common_keywords.append(word1 if len(word1) > len(word2) else word2)
    if not common_keywords:
        combined_keywords = set(keywords1 + keywords2)
        return list(combined_keywords)
    return list(set(common_keywords))

def fetch_wikipedia_content(keyword):
    URL = "https://en.wikipedia.org/w/api.php"
    search_params = {
        "action": "query",
        "list": "search",
        "srsearch": keyword,
        "format": "json"
    }
    search_response = requests.get(URL, params=search_params)
    search_data = search_response.json()
    if search_data["query"]["search"]:
        first_result_title = search_data["query"]["search"][0]["title"]
    else:
        print("No results found. Please try a different search term.",flush=True)
        return -1
    content_params = {
        "action": "query",
        "format": "json",
        "titles": first_result_title,
        "prop": "extracts",
        "explaintext": True,
    }

    content_response = requests.get(URL, params=content_params)
    content_data = content_response.json()

    page = next(iter(content_data['query']['pages'].values()))
    if 'extract' in page:
        content = page['extract']

        unwanted_sections = ["See also", "References", "Sources", "Further reading", "External links"]
        cleaned_content = content
        for section in unwanted_sections:
            section_pos = cleaned_content.find(f"\n== {section} ==")
            if section_pos != -1:
                cleaned_content = cleaned_content[:section_pos]

        return cleaned_content
    else:
        print("Content not found for the fetched article.")
        return -1

In [ ]:
## Read the pipeline1's output csv file
df = pd.read_csv('case_1_test_set_predictions.csv')
df = df.dropna(subset=['gpt_4_1106_preview_error_flag'])
df['gpt_4_1106_preview_sentence_id'] = df['gpt_4_1106_preview_sentence_id'].astype(int)
df['gpt_4_1106_preview_error_flag'] = df['gpt_4_1106_preview_error_flag'].apply(convert_value)
print("ERROR FLAG: ",df['gpt_4_1106_preview_error_flag'].value_counts(dropna=False))
df.reset_index(inplace=True)
print("df.shape: ",df.shape)

## Extract only the positive predictions by gpt-4 api
df_fp = df[df['gpt_4_1106_preview_error_flag'] ==1]
df_fp.reset_index(inplace=True)
modified_df = apply_models(df_fp, 'gpt_4_1106_preview_Error_Sentence')

keywords_to_remove = ["patient", "ordered", "evaluate", "diagnosed", "diagnosis", "diagnosis'","administered", "examining", "physical examination", "exam", "findings", "evaluation", "scheduled", "patient's", "suspected", "prescribed", "observation",
                      "infected", "infection", "plan", "labs", "presence", "smear", "symptoms'", "symptoms","cause", "patient's infection", "organism", "against", "laboratory studies", "causal pathogen", "casual", "pathology report"]
# Need to replace "cultures" with "blood cultures"
modified_df['en_core_sci_scibert'] = modified_df['en_core_sci_scibert'].apply(remove_keywords)
modified_df['en_ner_bc5cdr_md'] = modified_df['en_ner_bc5cdr_md'].apply(remove_keywords)

modified_df['Common_Keywords_Longest'] = modified_df.apply(find_longest_common_keywords, axis=1)
modified_df['final_keyword'] = modified_df['Common_Keywords_Longest'].apply(find_longest_keyword)
# modified_df

modified_df['reevaluated_gpt_4_1106_preview_error_flag'] = 0
modified_df['reevaluated_gpt_4_1106_preview_Corrected_Sentence'] = ''

max_retries = 3
default_wait_time = 6  # Default wait time in seconds

for i, row in modified_df.iterrows():
    print("#####################################")
    time.sleep(15)
    if i % 5 == 0:
        time.sleep(10)
        
    retry_count = 0
    while retry_count < max_retries:
        try:
            print(f"Currently accessing Test Set id: {i}", flush=True)
            text = row['Text']
            gpt_suspicious_sentence = row['gpt_4_1106_preview_Error_Sentence']
            disease_keyword_context = fetch_wikipedia_content(row['final_keyword'])
            if(disease_keyword_context==-1):
                print(f"No wiki page found for id: {row['Text ID']}",flush=True)
                message_text = [{"role":"system","content":
                  f"""I need your assistance in re-evaluating a sentence for medical accuracy within a clinical context. Take for example:
                  Example 1:
                    Clinical Text: ```A 17-year-old boy comes to the physician because of body aches and sore throat for 1 week. He has no history of serious illness and takes no medications. He lives with his parents; they recently adopted a cat from an animal shelter. He is sexually active with one female partner, and they use condoms consistently. His temperature is 38.7 C (101.7 F), pulse is 99/min, and blood pressure is 110/72 mm Hg. Examination shows bilateral posterior cervical lymphadenopathy. The pharynx is red and swollen. Laboratory studies show:
                      Hemoglobin 15 g/dL
                      Leukocyte count 11,500/mm3
                      Segmented neutrophils 48%
                      Band forms 2%
                      Basophils 0.5%
                      Eosinophils 1%
                      Lymphocytes 45%
                      Monocytes 3.5%
                      When the patient's serum is added to a sample of horse erythrocytes, the cells aggregate together. The causal pathogen is cytomegalovirus.```
                    sentence identified for re-evaluation is: ```The causal pathogen is cytomegalovirus.```
                    Error Flag: yes
                    Revised Sentence: The causal pathogen is Epstein-Barr virus.
                    
                    Example 2:
                    Clinical Text: ```A previously healthy 18-year-old woman comes to the emergency department because of diarrhea and abdominal cramps since the previous evening. She has had around 3Ã¢â‚¬â€œ4 episodes of watery stools. She feels nauseous and has vomited twice. She recollects eating out 2 days ago. She has been on a vegan diet for 6 months. She takes no medications and has not traveled anywhere recently. Her temperature is 36.8 (98.2 F), pulse is 73/min, and blood pressure is 110/70 mm Hg. Examination shows dry mucous membranes. Abdominal examination is unremarkable. Norovirus was determined to be the casual organism.```
                    sentence identified for re-evaluation is: ```Norovirus was determined to be the casual organism.```
                    Error Flag: No
                    Revised Sentence: -1
                    
                    Example 3:
                    Clinical Text: ```A 5-year-old boy is brought to the emergency department by his grandmother because of difficulty breathing. Over the past two hours, the grandmother has noticed his voice getting progressively hoarser and occasionally muffled, with persistent drooling. He has not had a cough. The child recently immigrated from Africa, and the grandmother is unsure if his immunizations are up-to-date. He appears uncomfortable and is sitting up and leaning forward with his chin hyperextended. His temperature is 39.5 C (103.1 F), pulse is 110/min, and blood pressure is 90/70 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 95%. Pulmonary examination shows inspiratory stridor and scattered rhonchi throughout both lung fields, along with poor air movement. Pharyngoscopy is ordered. ```
                    sentence identified for re-evaluation is: ```Pharyngoscopy is ordered.```
                    Error Flag: Yes
                    Revised Sentence: Nasotracheal intubation is performed.
                    
                    Example 4:
                    Clinical Text:```A 59-year-old man comes to the emergency department because of excruciating left knee pain for 4 days. He underwent a total knee arthroplasty of his left knee joint 4 months ago. He has hypertension and osteoarthritis. Current medications include glucosamine, amlodipine, and meloxicam. His temperature is 38.1 C (100.6 F), pulse is 97/min, and blood pressure is 118/71 mm Hg. Examination shows a tender, swollen left knee joint; range of motion is limited by pain. Analysis of the synovial fluid confirmed septic arthritis and presence of Staphylococcus epidermidis, and the prosthesis was removed.```
                    sentence identified for re-evaluation is: ```Analysis of the synovial fluid confirmed septic arthritis and presence of Staphylococcus epidermidis, and the prosthesis was removed.```
                    Error Flag: No
                    Revised Sentence: -1
                          
                    Now it is your turn. Here's are the two inputs:
                    Clinical Text :```{text}'''
                    The sentence identified for re-evaluation is: ```{gpt_suspicious_sentence}```
                    Please evaluate the accuracy of this sentence in the context of the clinical information provided. Consider any potential inaccuracies or misinterpretations related to diagnoses, infection detection, or any other relevant medical details. Provide a detailed explanation of your assessment. If you find the sentence to be inaccurate, kindly suggest how it could be corrected or improved.
                    Also provide your final answer in json format with "Error Flag" (yes/no) and "Revised Sentence" (corrected sentence, -1 otherwise) as the two keys
                """
                 }]
            else:
                message_text = [{"role":"system","content":
                  f"""I need your assistance in re-evaluating a sentence for medical accuracy within a clinical context. Take for example:
                  Example 1:
                  Clinical Text: ```A 17-year-old boy comes to the physician because of body aches and sore throat for 1 week. He has no history of serious illness and takes no medications. He lives with his parents; they recently adopted a cat from an animal shelter. He is sexually active with one female partner, and they use condoms consistently. His temperature is 38.7 C (101.7 F), pulse is 99/min, and blood pressure is 110/72 mm Hg. Examination shows bilateral posterior cervical lymphadenopathy. The pharynx is red and swollen. Laboratory studies show:
                  Hemoglobin 15 g/dL
                  Leukocyte count 11,500/mm3
                  Segmented neutrophils 48%
                  Band forms 2%
                  Basophils 0.5%
                  Eosinophils 1%
                  Lymphocytes 45%
                  Monocytes 3.5%
                  When the patient's serum is added to a sample of horse erythrocytes, the cells aggregate together. The causal pathogen is cytomegalovirus.```
                sentence identified for re-evaluation is: ```The causal pathogen is cytomegalovirus.```
                Error Flag: yes
                Revised Sentence: The causal pathogen is Epstein-Barr virus.
                
                Example 2:
                Clinical Text: ```A previously healthy 18-year-old woman comes to the emergency department because of diarrhea and abdominal cramps since the previous evening. She has had around 3Ã¢â‚¬â€œ4 episodes of watery stools. She feels nauseous and has vomited twice. She recollects eating out 2 days ago. She has been on a vegan diet for 6 months. She takes no medications and has not traveled anywhere recently. Her temperature is 36.8 (98.2 F), pulse is 73/min, and blood pressure is 110/70 mm Hg. Examination shows dry mucous membranes. Abdominal examination is unremarkable. Norovirus was determined to be the casual organism.```
                sentence identified for re-evaluation is: ```Norovirus was determined to be the casual organism.```
                Error Flag: No
                Revised Sentence: -1
                
                Example 3:
                Clinical Text: ```A 5-year-old boy is brought to the emergency department by his grandmother because of difficulty breathing. Over the past two hours, the grandmother has noticed his voice getting progressively hoarser and occasionally muffled, with persistent drooling. He has not had a cough. The child recently immigrated from Africa, and the grandmother is unsure if his immunizations are up-to-date. He appears uncomfortable and is sitting up and leaning forward with his chin hyperextended. His temperature is 39.5 C (103.1 F), pulse is 110/min, and blood pressure is 90/70 mm Hg. Pulse oximetry on room air shows an oxygen saturation of 95%. Pulmonary examination shows inspiratory stridor and scattered rhonchi throughout both lung fields, along with poor air movement. Pharyngoscopy is ordered. ```
                sentence identified for re-evaluation is: ```Pharyngoscopy is ordered.```
                Error Flag: Yes
                Revised Sentence: Nasotracheal intubation is performed.
                
                Example 4:
                Clinical Text:```A 59-year-old man comes to the emergency department because of excruciating left knee pain for 4 days. He underwent a total knee arthroplasty of his left knee joint 4 months ago. He has hypertension and osteoarthritis. Current medications include glucosamine, amlodipine, and meloxicam. His temperature is 38.1 C (100.6 F), pulse is 97/min, and blood pressure is 118/71 mm Hg. Examination shows a tender, swollen left knee joint; range of motion is limited by pain. Analysis of the synovial fluid confirmed septic arthritis and presence of Staphylococcus epidermidis, and the prosthesis was removed.```
                sentence identified for re-evaluation is: ```Analysis of the synovial fluid confirmed septic arthritis and presence of Staphylococcus epidermidis, and the prosthesis was removed.```
                Error Flag: No
                Revised Sentence: -1
                      
                Now it is your turn. Here's are the two inputs and one wikipedia reference for the keyword:
                Clinical Text :```{text}'''
                The sentence identified for re-evaluation is: ```{gpt_suspicious_sentence}```
                Wikipedia Reference: ```{disease_keyword_context}```
                Please evaluate the accuracy of this sentence in the context of the clinical information provided. Consider any potential inaccuracies or misinterpretations related to diagnoses, infection detection, or any other relevant medical details. Provide a detailed explanation of your assessment. If you find the sentence to be inaccurate, kindly suggest how it could be corrected or improved.
                Also provide your final answer in json format with "Error Flag" (yes/no) and "Revised Sentence" (corrected sentence, -1 otherwise) as the two keys
                """
                 }]

            completion = openai.ChatCompletion.create(engine="",messages = message_text,
                          temperature=0, max_tokens=800,
                          top_p=0.95, frequency_penalty=0,
                          presence_penalty=0, stop=None)
            message_content = completion["choices"][0]["message"]["content"]
            error_flag_start = message_content.find('"Error Flag": ') + len('"Error Flag": ')
            error_flag_end = message_content.find(',', error_flag_start)
            error_flag_value = message_content[error_flag_start:error_flag_end].strip('"')
            revised_sentence_start = message_content.find('"Revised Sentence": ') + len('"Revised Sentence": ')
            revised_sentence_end = message_content.find('\n', revised_sentence_start)
            revised_sentence_value = message_content[revised_sentence_start:revised_sentence_end].strip()
            revised_sentence_value = int(revised_sentence_value) if revised_sentence_value.isdigit() or (revised_sentence_value.startswith('-') and revised_sentence_value[1:].isdigit()) else revised_sentence_value
            modified_df.at[i, 'reevaluated_gpt_4_1106_preview_error_flag'] = error_flag_value
            modified_df.at[i, 'reevaluated_gpt_4_1106_preview_Corrected_Sentence'] = revised_sentence_value
            modified_df.to_csv('EA_shared_task_outputs/Test_set_results/test_set_preds_with_RAG_temp.csv',index=False)
            break

        except openai.error.APIConnectionError as e:
            print(f"Timeout error encountered at index {i}, attempt {retry_count + 1}: {e}", flush=True)
            retry_count += 1
            time.sleep(default_wait_time)  # Wait before retrying

        except openai.error.InvalidRequestError as e:
            print(f"InvalidRequestError at index {i}: {e}", flush=True)
            break  # Skip to the next record

        except openai.error.RateLimitError as e:
            match = re.search(r"Please retry after (\d+) seconds", str(e))
            if match:
                wait_time = int(match.group(1)) + 1  # Add a 1-second buffer
            else:
                wait_time = default_wait_time
            print(f"RateLimitError at index {i}. Retrying after {wait_time} seconds.", flush=True)
            time.sleep(wait_time)
            retry_count += 1

        except openai.error.OpenAIError as e:
            print(f"Unexpected OpenAIError encountered at index {i}: {e}", flush=True)
            break  # Exit the retry loop on unexpected OpenAI error

        except Exception as e:
            print(f"Unexpected error encountered at index {i}: {e}", flush=True)
            break  # Exit the retry loop on any other unexpected error

modified_df.to_csv('test_set_preds_with_RAG_28_march.csv',index=False)
print("Final csv saved!!",flush=True)

## case 1 0's predictions

In [4]:
df = pd.read_csv('EA_shared_task_outputs/Test_set_results/case_1_test_set_predictions.csv')
df = df.dropna(subset=['gpt_4_1106_preview_error_flag'])
df['gpt_4_1106_preview_sentence_id'] = df['gpt_4_1106_preview_sentence_id'].astype(int)
df['gpt_4_1106_preview_error_flag'] = df['gpt_4_1106_preview_error_flag'].apply(convert_value)
print("ERROR FLAG: ",df['gpt_4_1106_preview_error_flag'].value_counts(dropna=False))
df.reset_index(inplace=True)
print("df.shape: ",df.shape)

## Extract only the positive predictions by gpt-4 api
df_0 = df[df['gpt_4_1106_preview_error_flag'] ==0]
df_0.reset_index(inplace=True)
print("df.shape: ",df_0.shape)
df_0.to_csv('EA_shared_task_outputs/Test_set_results/test_set_preds_0s_without_RAG.csv',index=False)

ERROR FLAG:  gpt_4_1106_preview_error_flag
1    663
0    262
Name: count, dtype: int64
df.shape:  (925, 8)
df.shape:  (262, 9)
